This notebook contains code to replicate most of the results in the [Macro Technical paper](https://www.bankofengland.co.uk/macro-technical-paper/2026/learning-from-forecast-errors-the-Banks-enhanced-approach-to-forecast-evaluation), using the `forecast_evaluation` package.

In [ ]:
import pandas as pd
import forecast_evaluation as fe

In [ ]:
def covid_filter_gdp(df: pd.DataFrame) -> pd.DataFrame:
    """Filter data to exclude COVID-affected periods.

    For 'gdpkp' variable: excludes dates from 2020-01-01 to 2022-03-31 unless the
    forecast vintage is from 2022-01-01 onwards.
    For other variables: removes all 2020 and 2021 dates for pre-2020Q4 vintages.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing forecast data with 'variable', 'date', and 'vintage_date_forecast' columns

    Returns
    -------
    pd.DataFrame
        Filtered DataFrame with COVID periods removed based on variable type
    """
    # Apply GDP-specific filter for gdpkp rows
    gdpkp_mask = df["variable"] == "gdpkp"
    gdpkp_filter = (
        (df["date"] < "2020-01-01") | (df["date"] >= "2022-04-01") | (df["vintage_date_forecast"] >= "2022-01-01")
    )

    # Combine filters: use GDP filter for gdpkp rows, default filter for others
    df = df[(gdpkp_mask & gdpkp_filter) | (~gdpkp_mask)]

    return df

# Load data

In [ ]:
data = fe.ForecastData(load_fer=True)
data = data.filter(custom_filter=fe.filter_fer_variables)

# Create a copy of the data for 2015 onwards, without COVID filter
data_2015 = data.copy().filter(start_vintage="2015-01-01")

# Create a copy of the data for 2015 onwards, with COVID filter for GDP
data_2015_covid_filter = data.copy().filter(start_vintage="2015-01-01", custom_filter=covid_filter_gdp)

# Hedgehog

In [ ]:
fe.plot_hedgehog(data=data, variable="aweagg", forecast_source="mpr", metric="yoy", convert_to_percentage=True)
fe.plot_hedgehog(data=data, variable="cpisa", forecast_source="mpr", metric="yoy", convert_to_percentage=True)
fe.plot_hedgehog(data=data, variable="gdpkp", forecast_source="mpr", metric="yoy", convert_to_percentage=True)
fe.plot_hedgehog(data=data, variable="unemp", forecast_source="mpr", metric="levels", convert_to_percentage=True)

# Forecast Density

In [ ]:
fe.plot_forecast_error_density(
    data=data,
    variable="cpisa",
    horizon=4,
    metric="yoy",
    frequency="Q",
    source="mpr",
    k=12,
    highlight_dates=pd.date_range(start="2022-01-01", end="2024-12-31", freq="QE"),
)

# Forecast errors

In [ ]:
fe.plot_errors_across_time(
    data=data.copy().filter(start_date="2015-01-01"),
    variable="aweagg",
    metric="yoy",
    frequency="Q",
    error="raw",
    horizons=[0, 4, 8],
    sources=["mpr", "baseline ar(p) model"],
    k=12,
    ma_window=4,
    convert_to_percentage=True,
)

In [ ]:
existing_plot = fe.plot_errors_across_time(
    data=data,
    variable="gdpkp",
    metric="yoy",
    error="raw",
    horizons=[0],
    sources="mpr",
    frequency="Q",
    k=0,
    ma_window=4,
    convert_to_percentage=True,
    return_plot=True,
    custom_labels={"mpr": "k=0"},
)

fe.plot_errors_across_time(
    data=data,
    variable="gdpkp",
    metric="yoy",
    frequency="Q",
    error="raw",
    horizons=[0],
    sources="mpr",
    k=12,
    ma_window=4,
    convert_to_percentage=True,
    existing_plot=existing_plot,
    custom_labels={"mpr": "k=12"},
)

## Accuracy

In [ ]:
# Run the accuracy analysis
df_accuracy = fe.compute_accuracy_statistics(data=data_2015_covid_filter, k=12)

# Produce accuracy charts
df_accuracy.plot(variable="aweagg", metric="yoy")
df_accuracy.plot(variable="cpisa", metric="yoy")
fe.plot_accuracy(
    df=df_accuracy[df_accuracy["source"] != "baseline random walk model"],
    variable="gdpkp",
    metric="yoy",
)
df_accuracy.plot(variable="unemp", metric="levels")

### Diebold-Mariano

In [ ]:
# Run Diebold-Mariano analysis
df_dm = fe.diebold_mariano_table(data=data_2015, benchmark_model="mpr")

## Bias

In [ ]:
# Run the bias analysis
df_bias = fe.bias_analysis(data=data_2015_covid_filter, source="mpr", k=12)

# Produce bias charts
df_bias.plot(variable="aweagg", metric="yoy")
df_bias.plot(variable="cpisa", metric="yoy")
df_bias.plot(variable="gdpkp", metric="yoy")
df_bias.plot(variable="unemp", metric="levels")

# Rolling bias

In [ ]:
# Run the rolling bias with fluctuations test
rolling_bias = fe.fluctuation_tests(
    data=data.copy().filter(variables=["aweagg"], sources=["mpr"], metrics=["yoy"]),
    window_size=16,
    test_func=fe.bias_analysis,
    test_args={"k": 12},
)

In [ ]:
rolling_bias.plot(horizons=[0, 4, 8], variable="aweagg", source="mpr", convert_to_percentage=True)

## Blanchard-Leigh regressions

In [ ]:
bl_results = fe.blanchard_leigh_horizon_analysis(
    data=data,
    source="mpr",
    outcome_variable="cpisa",
    outcome_metric="yoy",
    instrument_variable="gdpkp",
    instrument_metric="yoy",
)

bl_results.plot()

## Weak Efficiency

### Optimal scaling

In [ ]:
# Run the optimal scaling analysis
df_optimal_scaling = fe.weak_efficiency_analysis(data=data_2015_covid_filter, k=12)

### Correlation of revisions & forecast errors 

In [ ]:
# Run the test for correlation between revisions and forecast errors
df_revisions_errors_correlation = fe.revisions_errors_correlation_analysis(data=data_2015, k=12)

### Revision predictability

In [ ]:
# Run the revision predictability analysis
df_revision_predictability = fe.revision_predictability_analysis(data=data_2015, n_revisions=5)